In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = '1'

In [2]:
import sys
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from PIL import Image

from time import time, sleep
from datetime import datetime

import json

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import torchvision.utils
from torchvision import models
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torchsummary import summary

from sklearn.metrics import f1_score
import timm

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [4]:
class ImageDataset(Dataset):
    def __init__(self, data_list, transform=None):
        self.data_list = data_list
        self.transform = transform
    
    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, idx):
        data_dict = self.data_list[idx]
        img_path_1 = data_dict['img1']
        
        img_1 = Image.open(img_path_1).convert("RGB")
        img_1 = self.transform(img_1)
        
        return img_1

In [5]:
transforms = transforms.Compose([
    transforms.ToTensor(),
])

In [6]:
fold = []

path = './Homework Dataset'
for idx, it in enumerate(os.listdir(path)):
    full_paths = []
    semi_full = os.path.join(os.path.join(path, it), 'images')
    for it2 in sorted(os.listdir(semi_full)):
        full_path = os.path.join(semi_full, it2)
        full_paths.append(full_path)
    for x in range(len(full_paths)):
        fold.append({"img1": full_paths[x]})
            
print(len(fold))

1423


In [7]:
ds = ImageDataset(fold, transform=transforms)

In [8]:
def dataset_statistics(dataset):
    mean = [0, 0, 0]
    std = [0, 0, 0]

    std[0], mean[0] = torch.std_mean(torch.cat([dataset[i][0][0].flatten() for i in range(len(dataset))]))
    std[1], mean[1] = torch.std_mean(torch.cat([dataset[i][0][1].flatten() for i in range(len(dataset))]))
    std[2], mean[2] = torch.std_mean(torch.cat([dataset[i][0][2].flatten() for i in range(len(dataset))]))

    return mean, std

mean, std = dataset_statistics(ds)
mean, std

([tensor(0.4654), tensor(0.4655), tensor(0.4654)],
 [tensor(0.2362), tensor(0.2349), tensor(0.2349)])